In [19]:
pip install requests bs4 pathlib numpy xarray boto3 dotenv

Note: you may need to restart the kernel to use updated packages.


In [20]:
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import xarray as xr
import numpy as np
import json
from datetime import datetime
import boto3

In [21]:
fecha_hoy = datetime.now().strftime("%Y%m%d")
horas_validas = ["00", "06", "12", "18"]

base_url = f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.{fecha_hoy}/"
html = requests.get(base_url).text
soup = BeautifulSoup(html, "html.parser")

hora_disponible = None
for h in reversed(horas_validas):
    if soup.find("a", href=f"{h}/"):
        hora_disponible = h
        break

if not hora_disponible:
    raise Exception("No se encontró hora válida para hoy.")

print(f"Hora más reciente disponible: {hora_disponible}Z")

Hora más reciente disponible: 12Z


In [22]:

filename = f"gfs.t{hora_disponible}z.atmf000.nc"
download_url = f"{base_url}{hora_disponible}/atmos/{filename}"
local_path = filename

if not Path(local_path).exists():
    print("Descargando archivo...")
    with requests.get(download_url, stream=True) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print("Descarga completa.")
else:
    print("Archivo ya existe, no se descarga de nuevo.")

Archivo ya existe, no se descarga de nuevo.


In [23]:
ds = xr.open_dataset(local_path)

u = ds['ugrd'].isel(time=0, pfull=-1)
v = ds['vgrd'].isel(time=0, pfull=-1)
lons = ds['lon'].values
lats = ds['lat'].values

u_vals = u.values
v_vals = v.values

features = []
step = 20  

for i in range(0, lats.shape[0], step):
    for j in range(0, lats.shape[1], step):
        lon = float(lons[i][j])
        lat = float(lats[i][j])
        u_val = float(u_vals[i][j])
        v_val = float(v_vals[i][j])
        speed = np.sqrt(u_val**2 + v_val**2)
        direction = np.arctan2(v_val, u_val) * 180 / np.pi

        features.append({
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [lon, lat]},
            "properties": {
                "u": u_val,
                "v": v_val,
                "speed": speed,
                "direction": direction
            }
        })

geojson = {
    "type": "FeatureCollection",
    "features": features
}

geojson_filename = "wind.geojson"
with open(geojson_filename, "w") as f:
    json.dump(geojson, f)

print("GeoJSON creado: wind.geojson ✅")



GeoJSON creado: wind.geojson ✅


In [24]:
from dotenv import load_dotenv
import os

load_dotenv() 

s3_bucket = os.getenv("S3_BUCKET")
s3_key = f"wind/{fecha_hoy}/wind_{hora_disponible}Z.geojson"  
s3 = boto3.client("s3")

try:
    s3.upload_file(geojson_filename, s3_bucket, s3_key)
    print(f"Archivo subido a S3: s3://{s3_bucket}/{s3_key} ✅")
except Exception as e:
    print(f"Error al subir a S3: {e}")

nomdas-data
Error al subir a S3: Unable to locate credentials
